КарезинМА_МО_ДЗ 2
Задание1

In [ ]:
from math import sqrt
from tabulate import tabulate
import sympy

columns = ['mu', 'x1', 'x2', 'g1', 'g2', 'a11', 'a12', 'a21', 'a22', 'b1', 'b2', 'f', 'F', 'Штр. Слог.']
rows = []

x1, x2 = sympy.symbols('x1 x2', real=True)

n = 16
f = x1**2 + x2**2 - (n+5)*x1 - (2*n + 10/n)*x2
print("Начальная функция:", f, '\n')

def evaluate(f, x):
    return f.subs([(x1, x[0]), (x2, x[1])])

def gradient(f, x):
    return [evaluate(f.diff(x1), x), evaluate(f.diff(x2), x)]

def find_lambda(phi, lambda_symbol):
    phi_derivative = phi.diff(lambda_symbol)
    solutions = sympy.solve(phi_derivative, lambda_symbol)

    phi_second_derivative = phi_derivative.diff(lambda_symbol)
    valid_solutions = [float(sol) for sol in solutions if float(phi_second_derivative.subs([(lambda_symbol, sol)])) > 0]
    return valid_solutions[0]

def find_minimum(f, x):
    delta_threshold = 0.001
    lambda_symbol = sympy.symbols('lamb')

    while True:
        gradient_values = gradient(f, x)
        if all(abs(val) < 0.001 for val in gradient_values):
            return x

        phi = f.subs([
            (x1, x[0] - gradient_values[0]*lambda_symbol),
            (x2, x[1] - gradient_values[1]*lambda_symbol)
            ])
        lambda_result = find_lambda(phi, lambda_symbol)

        next_x = [x[0] - gradient_values[0]*lambda_result, x[1] - gradient_values[1]*lambda_result]
        delta = sqrt((next_x[0] - x[0])**2 + (next_x[1] - x[1])**2)
        x = next_x

        if delta < delta_threshold:
            return x

x_init = find_minimum(f, [1, 1])
print(f"Начальное приближение: {x_init[0]} {x_init[1]}")
print()

mu = sympy.symbols('mu')
cur_mu = 0.1

limitation1 = 9*x1 + 8*x2 - 72
limitation2 = x1 + 2*x2 - 10

f2 = f+mu*(sympy.Piecewise((0, limitation1<0), (limitation1**2, True)) + sympy.Piecewise((0, limitation2<0), (limitation2**2, True)))
phi1 = f2.diff(x1)
phi2 = f2.diff(x2)
a11 = phi1.diff(x1)
a12 = phi1.diff(x2)
a21 = phi2.diff(x1)
a22 = phi2.diff(x2)

b1 = -phi1 + a11*x1 + a12*x2
b2 = -phi2 + a21*x1 + a22*x2

eps_threshold = 0.0001
beta = 10

phi1 = f2.diff(x1)
phi2 = f2.diff(x2)

def evaluate_value(f_func, x_values, mu_value):
    return f_func.subs([(x1, x_values[0]), (x2, x_values[1]), (mu, mu_value)])

def build_iteration():
    ret = [cur_mu]
    ret.extend(x) # x1, x2
    ret.extend([ # g1, g2
        evaluate(limitation1, x),
        evaluate(limitation2, x)
    ])
    a11_res = evaluate_value(a11, x, cur_mu)
    a12_res = evaluate_value(a12, x, cur_mu)
    a21_res = evaluate_value(a21, x, cur_mu)
    a22_res = evaluate_value(a22, x, cur_mu)
    b1_res = evaluate_value(b1, x, cur_mu)
    b2_res = evaluate_value(b2, x, cur_mu)

    ret.extend([a11_res, a12_res, a21_res, a22_res, b1_res, b2_res])
    f_res = evaluate(f, x)
    f2_res = evaluate_value(f2, x, cur_mu)
    cur_eps = f2_res-f_res
    ret.extend([f_res, f2_res, cur_eps])

    delta0 = a11_res*a22_res - a12_res*a21_res
    delta1 = b1_res*a22_res - a12_res*b2_res
    delta2 = a11_res*b2_res - b1_res*a21_res
    next_x = [delta1/delta0, delta2/delta0]

    return [round(i, 5) for i in ret], next_x, cur_eps

x = x_init

while True:
    cur_iter, next_x, cur_eps = build_iteration()
    rows.append(cur_iter)
    if cur_eps < eps_threshold:
        break
    x = next_x
    cur_mu *= beta

print("Оптимальное значение и точка после метода штрафных функций:")
print("Оптимальное значение f:", cur_iter[-3])
print(f"Оптимальная точка: {x[0]} {x[1]}")
print()

print("Таблица итераций:")
print(tabulate(rows, headers=columns, tablefmt='orgtbl'))

Начальная функция: x1**2 - 21*x1 + x2**2 - 32.625*x2 

Начальное приближение: 10.5000000000000 16.3125000000000

Оптимальное значение и точка после метода штрафных функций:
Оптимальное значение f: -156.89454
Оптимальная точка: 3.87500013250000 3.06250026374999

Таблица итераций:
|         mu |       x1 |       x2 |          g1 |           g2 |        a11 |        a12 |        a21 |        a22 |               b1 |               b2 |        f |         F |     Штр. Слог. |
|------------+----------+----------+-------------+--------------+------------+------------+------------+------------+------------------+------------------+----------+-----------+----------------|
|      0.1   | 10.5     | 16.3125  | 153         | 33.125       |     18.4   |     14.8   |     14.8   |     15.6   |    152.6         |    151.825       | -376.348 | 2074.28   | 2450.63        |
|      1     |  1.96397 |  7.86912 |   8.62868   |  7.70221     |    166     |    148     |    148     |    138     |   1337        

Задание 2

In [ ]:
columns = ['mu', 'x1', 'x2', 'f', 'F', 'eps', 'g']
rows = []

# функция f
n = 16
f = x1**2 + x2**2 - (n+5)/2*x1 - (n+5)/4*x2
limitation1 = x1 + x2 - 1

x_init = [0, 0]

gamma = 0.25
mu_init = 10
eps_threshold = 0.05

# функция F
f2 = f - mu/limitation1

rows.append([' ', x_init[0], x_init[1], evaluate(f, x_init), ' ', ' ', evaluate(limitation1, x_init)])

# Метод Ньютона

def newton_find_mu(f, x):
    phi1 = f.diff(x1)
    phi2 = f.diff(x2)
    mu1, mu2 = sympy.symbols('mu1 mu2')
    a11 = evaluate(phi1.diff(x1), x)
    a12 = evaluate(phi1.diff(x2), x)
    a21 = evaluate(phi2.diff(x1), x)
    a22 = evaluate(phi2.diff(x2), x)
    b1 = evaluate(-phi1, x)
    b2 = evaluate(-phi2, x)

    found_solution = sympy.solve([a11*mu1 + a12*mu2 - b1, a21*mu1 + a22*mu2 - b2], mu1, mu2)
    return [found_solution[mu1], found_solution[mu2]]

def find_minimum(f, x):
    delta_threshold = 0.001

    while True:
        mu1, mu2 = newton_find_mu(f, x)
        new_x = [x[0] + mu1, x[1] + mu2]
        delta = sqrt((new_x[0] - x[0])**2 + (new_x[1] - x[1])**2)
        if delta < delta_threshold:
            return new_x
        x = new_x

def new_iteration(f, og_f, it_mu, it_x):
    f_mu = f.subs([(mu, it_mu)])
    new_x = find_minimum(f_mu, it_x)
    new_eps = evaluate(it_mu/limitation1, new_x)
    return [round(i, 5) for i in [
        it_mu,
        new_x[0], new_x[1],
        evaluate(og_f, new_x),
        evaluate(f_mu, new_x),
        new_eps,
        evaluate(limitation1, new_x),
    ]], new_x, new_eps

print("Работа метода началась")

cur_x = x_init
cur_mu = mu_init

while True:
    new_iter, new_x, new_eps = new_iteration(f2, f, cur_mu, cur_x)
    rows.append(new_iter)
    cur_x = new_x
    cur_mu *= gamma
    if abs(new_eps) < eps_threshold:
        break
print()

print(tabulate(rows, headers=columns, tablefmt='orgtbl'))
print()

print("Миниальное значение функции f с указанным условием:", round(evaluate(f, cur_x), 4))
print("Точка минимума функции f с указанным условием:", cur_x)
print()

Работа метода началась

| mu      |      x1 |       x2 |         f | F         | eps      |           g |
|---------+---------+----------+-----------+-----------+----------+-------------|
|         | 0       |  0       |   0       |           |          | -1          |
| 10      | 1.25326 | -1.37174 |  -2.50519 | 6.43544   | -8.94063 | -1.11849    |
| 2.5     | 2.12893 | -0.49607 | -14.9709  | -18.92129 | 3.95036  |  0.63285    |
| 0.625   | 1.96676 | -0.65824 | -12.8937  | -14.91960 | 2.02585  |  0.30851    |
| 0.15625 | 1.73793 | -0.88707 |  -9.78385 | -8.73618  | -1.04767 | -0.14914    |
| 0.03906 | 1.8504  | -0.7746  | -11.3385  | -11.85390 | 0.51536  |  0.0757999  |
| 0.00977 | 1.79371 | -0.83129 | -10.5612  | -10.30138 | -0.25985 | -0.03758    |
| 0.00244 | 1.82192 | -0.80308 | -10.9496  | -11.07925 | 0.12965  |  0.0188301  |
| 0.00061 | 1.80779 | -0.81721 | -10.7555  | -10.69071 | -0.06482 | -0.00942039 |
| 0.00015 | 1.81486 | -0.81014 | -10.8527  | -10.88508 | 0.03238  |  0.004